In [3]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px

/Users/shazeb.asad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [32]:
# define a few things (project id, start date etc.)
project_id = "logistics-customer-staging"
client = bigquery.Client(project = project_id)
start_date = '2024-01-01'
end_date = '2024-03-01'

query = f"""
SELECT
    o.country_name,
    o.entity_id,
    o.region,
    ROUND(sum(revenue_eur),2) revenue_eur,
    ROUND(sum(delivery_fee_eur),2) delivery_fee_eur,
    ROUND(sum(front_margin_eur),2) front_margin_eur,
    ROUND(sum(joker_vendor_fee_eur),2) joker_vendor_fee_eur,
    ROUND(sum(service_fee_eur),2) service_fee_eur,
    ROUND(sum(mov_customer_fee_eur),2) mov_customer_fee_eur,
    ROUND(sum(priority_fee_eur),2) priority_fee_eur,
    ROUND(sum(commission_eur),2) commission_eur
FROM
`fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` o
where created_date between \"""" + start_date + """\" and \"""" + end_date + """\"
      and SUBSTR(entity_id, 1, STRPOS(entity_id, '_') - 1) not in ('ODR', 'GV')
group by 1,2,3
"""


test = client.query(query).to_dataframe()

/Users/shazeb.asad/Library/Python/3.9/lib/python/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/shazeb.asad/Library/Python/3.9/lib/python/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [34]:
test.to_csv('global_breakdown.csv', index=False)  

In [23]:
# Extract the part of the string before the underscore
test['entity'] = test['entity_id'].str.split('_').str[0]

entities_to_remove = ['ODR', 'GV']

# Filter the DataFrame based on the first two letters of the extracted string
filtered_df = test[~test['entity'].isin(entities_to_remove)]

filtered_df

,country_name,entity_id,region,entity
1,None,TB_JO,None,TB
4,None,FP_KH,None,FP
5,None,NP_HU,Europe,NP
6,None,FP_MM,Asia,FP
7,None,DJ_CZ,None,DJ
...,...,...,...,...
191,Turkey,YS_TR,MENA,YS
192,None,FP_TH,None,FP
193,Myanmar,FP_MM,Asia,FP
194,None,HF_EG,MENA,HF


In [18]:
test

,country_name,entity_id,region,entity
0,Bosnia and Herzegovina,GV_BA,Europe,GV
1,None,TB_JO,None,TB
2,Malaysia,ODR_MY,Asia,ODR
3,Cyprus,ODR_CY,Europe,ODR
4,None,FP_KH,None,FP
...,...,...,...,...
192,None,FP_TH,None,FP
193,Myanmar,FP_MM,Asia,FP
194,None,HF_EG,MENA,HF
195,Singapore,FP_SG,Asia,FP


In [ ]:
ODR, GV